In [2]:
import datetime
import logging
import sys
import time

import pandas as pd

from tigeropen.common.consts import BarPeriod, SecurityType, Market, Currency ,Language, QuoteRight, TradingSession, Market, SortDirection
from tigeropen.common.util.contract_utils import stock_contract
from tigeropen.common.util.order_utils import limit_order
from tigeropen.quote.quote_client import QuoteClient
from tigeropen.tiger_open_config import get_client_config, TigerOpenClientConfig
from tigeropen.common.util.signature_utils import read_private_key
from tigeropen.trade.trade_client import TradeClient
from datetime import datetime
from tigeropen.quote.domain.filter import OptionFilter, StockFilter, SortFilterData
from tigeropen.common.consts.filter_fields import StockField, AccumulateField, FinancialField, MultiTagField, \
    FinancialPeriod, AccumulatePeriod

client_logger = logging.getLogger('client')
client_logger.setLevel(logging.WARNING)
client_logger.addHandler(logging.StreamHandler(sys.stdout))
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# PRIVATE_KEY_PATH = "your private key path"
# TIGER_ID = "your tiger id"
# ACCOUNT = "your account"
# client_config = get_client_config(private_key_path=PRIVATE_KEY_PATH, tiger_id=TIGER_ID, account=ACCOUNT)
# quote_client = QuoteClient(client_config, logger=client_logger)
# trade_client = TradeClient(client_config, logger=client_logger)


def get_client_config():
    """
    https://quant.itigerup.com/#developer 开发者信息获取
    """
    is_sandbox = False
    # 港股牌照需用 props_path 参数指定token路径，如 '/Users/xxx/xxx/', 如不指定则取当前路径
    # 必须使用关键字参数指定 props_path
    client_config = TigerOpenClientConfig(props_path=r'C:\Users\SnowWinter\Desktop\ML\Quant\props')
    return client_config



# 调用上方定义的函数生成用户配置ClientConfig对象
client_config = get_client_config()

# 随后传入配置参数对象来初始化QuoteClient
quote_client = QuoteClient(client_config)

# 完成初始化后，就可以调用quote_client方法来使用调用QuoteClient对象的get_stock_brief方法来查询股票行情了
# 此处以美国股票为例，关于其他支持的市场及标的类型，请参考文档的基本操作部分。
# 对于使用多台设备调用API的用户，需先调用grab_quote_permission进行行情权限的抢占，详情请见基本操作-行情类-通用-grab_quote_permission方法说明
permissions = quote_client.grab_quote_permission() 

#输出list类型的行情权限权限列表
print(permissions)

# 调用API查询股票行情
stock_price = quote_client.get_stock_briefs(['00700'])

# 查询行情函数会返回一个包含当前行情快照的pandas.DataFrame对象，见返回示例。具体字段含义参见get_stock_briefs方法说明
print(stock_price)

def test_market_scanner():

    # # 股票基本数据过滤(is_no_filter为True时表示不启用该过滤器)
    # base_filter1 = StockFilter(StockField.FloatShare, filter_min=1e7, filter_max=1e13, is_no_filter=True)
    # base_filter2 = StockFilter(StockField.MarketValue, filter_min=1e8, filter_max=1e14, is_no_filter=False)
    # # 按财报时间过滤
    # base_filter3 = StockFilter(StockField.EarningDate, filter_min=int(datetime.strptime('2021-01-01', '%Y-%m-%d').timestamp() * 1000),
    #                                filter_max=int(datetime.strptime('2023-12-31', '%Y-%m-%d').timestamp() * 1000)
    #                                , is_no_filter=False)
    
    TTM_filter = StockFilter(FinancialField.TTM_PB,filter_min=0,filter_max=1,is_no_filter=False,financial_period=FinancialPeriod.LTM)
    ROA_filter = StockFilter(AccumulateField.ROA,filter_min=1,is_no_filter=False,accumulate_period=AccumulatePeriod.Five_Days)
    sort_field_data = SortFilterData(AccumulateField.ROA, sort_dir=SortDirection.DESC, period=AccumulatePeriod.Five_Days)
    # 请求的开始页码
    begin_page = 0
    page_size = 50
    # 是否为最后一页数据
    is_last_page = False
    # 筛选后的symbol列表
    scanner_result_symbols = set()

    while not is_last_page:
        # filters参数里填需要使用的过滤器
        result = quote_client.market_scanner(market=Market.US,
                                             filters=[TTM_filter, ROA_filter
                                                      ],
                                             sort_field_data=sort_field_data,
                                             page=begin_page,
                                             page_size=page_size)
        print(result)
        if result.total_page:
            for item in result.items:
                # item的类型为 ScannerResultItem 
                symbol = item.symbol
                market = item.market
                # 可以字典的形式获取某个filter的字段对应的值
                base_filter1_value = item[base_filter1]
                accumulate_filter_value = item[accumulate_filter]
                print(
                    f'page:{result.page}, symbol:{symbol}, base_filter1 value:{base_filter1_value}, accumulate_filter value:{accumulate_filter_value}')
            print(f'current page symbols:{result.symbols}')
            scanner_result_symbols.update(result.symbols)
        time.sleep(1)
        # 处理分页
        if result.page >= result.total_page - 1:
            is_last_page = True
        else:
            begin_page = result.page + 1

    print(f'scanned symbols:{scanner_result_symbols}')


[{'name': 'hkStockQuoteLv2', 'expire_at': -1}, {'name': 'HKEXFuturesQuoteLv1', 'expire_at': 1737971841000}, {'name': 'aStockQuoteLv1', 'expire_at': -1}, {'name': 'CBOTFuturesQuoteLv2', 'expire_at': 1737971841000}, {'name': 'CMEFuturesQuoteLv2', 'expire_at': 1737971841000}, {'name': 'COMEXFuturesQuoteLv2', 'expire_at': 1737971841000}, {'name': 'hkStockQuoteLv2Global', 'expire_at': 1737971841000}, {'name': 'CBOEFuturesQuoteLv2', 'expire_at': 1737971841000}, {'name': 'usStockQuoteLv2Totalview', 'expire_at': 1737971841000}, {'name': 'usQuoteBasic', 'expire_at': 1737971841000}, {'name': 'NYMEXFuturesQuoteLv2', 'expire_at': 1737971841000}]
  symbol   open   high    low  close  pre_close  latest_price    latest_time  ask_price  ask_size  bid_price  bid_size    volume  status  adj_pre_close
0  00700  430.6  431.4  419.2  421.0      430.8         421.0  1729498102027      421.0     67600      420.8     13900  16334100  NORMAL          430.8
